In [ ]:
%cd ..

# Bayesian optimization: finding the minimum time loss

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [ ]:
import pickle
import numpy as np
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from experimental_design import config
from sumo_grid_simulation.grid_simulation import Simulator

## Simulator

We start by initialising our simulation.

In [ ]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [ ]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * 50 / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route duration
        result.append(s['timeLoss']/s['duration'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [ ]:
gridSize = DiscreteParameter('gridSize', domain=[3, 4, 5, 6, 7, 8, 9, 10])
edgeMaxSpeed = ContinuousParameter('edgeMaxSpeed', min_value=8, max_value=20)  # 28.8 km/h to 72 km/h
numberOfLanes = DiscreteParameter('numberOfLanes', domain=[1, 2, 3])
accel = ContinuousParameter('accel', 1., 3.)

parameter_space = ParameterSpace([gridSize, edgeMaxSpeed, numberOfLanes, accel])
#parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 200 datapoints from the parameter space:

In [ ]:
#init_X, init_Y = pickle.load(open('experimental_design/init_points/timeloss_200_init_points.pkl', 'rb'))

design = RandomDesign(parameter_space)
num_init_points = 250
init_X = design.get_samples(num_init_points)
init_Y = user_function_time_loss(init_X)

To then fit a GP to these points

In [ ]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
emukit_model_random_init.optimize()

### Experimentally designed emulator

We also load the emulator obtained during experimental design:

In [ ]:
emukit_model_exp_design = pickle.load(open('experimental_design/models/experimental_design_20_init_points_200_loops_timeloss.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [ ]:
n_iter_bo = 40

### Acquisition functions

In [ ]:
acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [ ]:
acquisition_exp_design = ExpectedImprovement(emukit_model_exp_design)

### Optimisation loop

In [ ]:
bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
bo_random_init.run_loop(user_function_time_loss, n_iter_bo)

In [ ]:
bo_exp_design = BayesianOptimizationLoop(parameter_space, emukit_model_exp_design, acquisition=acquisition_exp_design)
bo_exp_design.run_loop(user_function_time_loss, n_iter_bo)

## Results

### Results on the random initialised emulator

In [ ]:
results_random_init = bo_random_init.get_results()

In [ ]:
results_random_init.minimum_location

In [ ]:
results_random_init.minimum_value*100

### Results on the experimentally designed emulator

In [ ]:
results_exp_design = bo_exp_design.get_results()

In [ ]:
results_exp_design.minimum_location

In [ ]:
results_exp_design.minimum_value*100